In [163]:
# One tone example

import spcm
from spcm import units
import numpy as np  
card : spcm.Card
with spcm.Card('/dev/spcm0') as card:
    card.card_mode(spcm.SPC_REP_STD_DDS)    # setup card for DDS
    # Setup the card
    channels = spcm.Channels(card)
    channels.enable(True)
    channels.output_load(50 * units.ohm)
    channels.amp(2.0 * units.V)
    card.write_setup() # IMPORTANT! this turns on the card's system clock signals, that are required for DDS to work
    
    # Setup DDS
    dds = spcm.DDS(card, channels=channels)
    dds.reset()
   
    # Switch groups of cores to other channels
    if len(channels) == 2:
        # pass
        dds.cores_on_channel(1, 
                             spcm.SPCM_DDS_CORE8,  spcm.SPCM_DDS_CORE9,  spcm.SPCM_DDS_CORE10, spcm.SPCM_DDS_CORE11, # Flex core block 8 - 11
                             spcm.SPCM_DDS_CORE20) # Fixed core 20
        
    temp = 360*np.random.rand()*units.degrees  
    
    # voltage_test_x = 1500
    # voltage_test_y = voltage_test_x*1.4
    
    # voltage amplitude 
    
    dds[0].amp(1900 * units.mV)
    # frequency and phase   
    dds[0].freq(97.5 * units.MHz) 
    dds[0].phase(temp)
    
    # dds[1].amp(100 * units.mV)
    # # frequency and phase   
    # dds[1].freq(60 * units.MHz)
    # dds[1].phase(temp)
    
    dds[8].amp(1600 * units.mV)
    # frequency and phase   
    dds[8].freq(97.5 * units.MHz)
    dds[8].phase(temp)
    
    dds.exec_at_trg()
    dds.write_to_card()

    # Start command including enable of trigger engine
    card.start(spcm.M2CMD_CARD_ENABLETRIGGER, spcm.M2CMD_CARD_FORCETRIGGER)

    input("Press Enter to Exit")
#aaa

In [ ]:
import spcm,time
from spcm import units
import numpy as np
# import matplotlib.pyplot as plt


def shape_linear(x):
    return x

def shape_cosine(x):
    return 0.5-0.5*np.cos(np.pi*x)

def shape_square(x):
    return np.where(x < 0.5, 0, 1)

# record the beginning time of the program  
start_time = time.time()
print(f"Program beginning time: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))}")

def calculate_slopes_for_given_shape(parameters,shape):
    frequency_range_Hz = parameters["endFreq_Hz"] - parameters["startFreq_Hz"]                      
    unit_slices_in = np.linspace(0, 1, num_segments+1, endpoint=True)
    unit_slices_out = shape(unit_slices_in)
    time_list = unit_slices_in*parameters["time_s"]         
    output_list = unit_slices_out*frequency_range_Hz+parameters["startFreq_Hz"]         
    t_diff = np.diff(time_list)
    y_diff = np.diff(output_list)
    return np.divide(y_diff, t_diff),time_list,output_list  


card : spcm.Card
with spcm.Card('/dev/spcm0') as card:       
    card.card_mode(spcm.SPC_REP_STD_DDS)    # setup card for DDS
    channels = spcm.Channels(card)
    channels.enable(True)
    channels.amp(1 * units.V)
    card.write_setup() # IMPORTANT! this turns on the card's system clock signals, that are required for DDS to work
    dds = spcm.DDS(card)    # Setup DDS
    dds.reset()
    dds.data_transfer_mode(spcm.SPCM_DDS_DTM_DMA)
    num_cores = len(dds)     # Start the DDS test
    first_init_freq_Hz  = 90 * units.MHz    # 20 Carriers from 90 to 110 MHz
    delta_init_freq_Hz  = 20*1/num_cores * units.MHz
    first_final_freq_Hz = 95  * units.MHz    # to 20 Carriers from 95 to 105 MHz
    delta_final_freq_Hz = 10*1/num_cores * units.MHz 
    
    # Ramp settings
    num_segments = 16
    total_time_s = 20.0 * units.s
    ramp_type = 'cosine' # 'cosine' or 'square' or '3rd-order'


    dds.freq_ramp_stepsize(1000)      # STEP 0 - Initialize frequencies
    dds.trg_timer(5.0 * units.s)
    dds.trg_src(spcm.SPCM_DDS_TRG_SRC_TIMER)
    for core in dds:
        core.amp(45 * units.percent / num_cores)
        core.freq(first_init_freq_Hz + int(core) * delta_init_freq_Hz)
    dds.exec_at_trg()
    dds.write_to_card()


    period_s = total_time_s / num_segments # seconds      # STEP 1 - Start the ramp
    dds.trg_timer(period_s) 
    
    parameters = []    # Define the parameters
    slopes = np.zeros((num_cores, num_segments))
    # plt.figure(figsize=(7,7))    # Show the results
    for core in dds:
        parameters = {
            "startFreq_Hz": first_init_freq_Hz + core.index * delta_init_freq_Hz, 
            "endFreq_Hz": first_final_freq_Hz + core.index * delta_final_freq_Hz, 
            "time_s": total_time_s, 
            }

        sl_core,time_list,output_list = calculate_slopes_for_given_shape(parameters,shape_cosine)
        slopes[core, :] = sl_core.to_base_units().magnitude

    #     plt.plot(time_list, output_list, 'ok')
    #     t_fine_s = np.linspace(time_list[0], time_list[1], 2, endpoint=True)
    #     for j, sl in enumerate(sl_core):
    #         plt.plot(t_fine_s + time_list[j], output_list[j] + sl*(t_fine_s), '--')

    # plt.show(block=False)

    # Do the slopes
    for j in range(num_segments):
        for core in dds:
            core.frequency_slope(slopes[core][j]) # Hz/s
        dds.exec_at_trg()

    # STEP 2 - Stop the ramp
    for core in dds:
        core.frequency_slope(0) # Hz/s
        core.freq(first_final_freq_Hz + core.index * delta_final_freq_Hz)
        core.phase(180*np.random.rand()*units.degrees)
    dds.exec_at_trg()
    dds.write_to_card()


    end_time = time.time()
    execution_time = end_time - start_time
    
    print(f"\nProgram ending time: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(end_time))}")
    print(f"Total time: {execution_time:.4f} s")
    # Start command including enable of trigger engine
    card.start(spcm.M2CMD_CARD_ENABLETRIGGER, spcm.M2CMD_CARD_FORCETRIGGER)

    input("Press Enter to Exit")

Program beginning time: 2025-09-30 17:55:50

Program ending time: 2025-09-30 17:55:50
Total time: 0.4773 秒


In [ ]:
import spcm
from spcm import units

card : spcm.Card
with spcm.Card('/dev/spcm0') as card:                         # if you want to open a specific card
# with spcm.Card('TCPIP::192.168.1.10::inst0::INSTR') as card:  # if you want to open a remote card
# with spcm.Card(serial_number=12345) as card:                  # if you want to open a card by its serial number
# with spcm.Card(card_type=spcm.SPCM_TYPE_AO) as card:             # if you want to open the first card of a specific type

    # setup card for DDS
    card.card_mode(spcm.SPC_REP_STD_DDS)

    # Setup the channels
    channels = spcm.Channels(card, card_enable=spcm.CHANNEL0)
    channels[0].enable(True)
    channels[0].output_load(50 * units.ohm)
    channels[0].amp(0.5 * units.V)
    card.write_setup() # IMPORTANT! this turns on the card's system clock signals, that are required for DDS to work
    
    # Setup DDS functionality
    dds = spcm.DDS(card, channels=channels)
    dds.reset()

    # Start the test
    # dds[0].amp(-20 * units.dBm)
    dds[0].amp(100 * units.mV)
    dds[0].freq(100 * units.MHz)
    dds[0].phase(20 * units.degrees)
    # Read back the exact frequency
    freq = dds[0].get_freq(return_unit=units.MHz)
    amp = dds[0].get_amp(return_unit=units.dBm)
    phase = dds[0].get_phase(return_unit=units.rad)
    print(f"Generated signal frequency: {freq} and amplitude: {amp} and phase: {phase}")
    
    dds.exec_at_trg()
    dds.write_to_card()

    # Start command including enable of trigger engine
    card.start(spcm.M2CMD_CARD_ENABLETRIGGER, spcm.M2CMD_CARD_FORCETRIGGER)

    input("Press Enter to Exit")


In [12]:
import spcm
import numpy as np  
from spcm import units


card : spcm.Card
# with spcm.Card('/dev/spcm0') as card:                         # if you want to open a specific card
# with spcm.Card('TCPIP::192.168.1.10::inst0::INSTR') as card:  # if you want to open a remote card
# with spcm.Card(serial_number=12345) as card:                  # if you want to open a card by its serial number
with spcm.Card(card_type=spcm.SPCM_TYPE_AO) as card:             # if you want to open the first card of a specific type

    # setup card for DDS
    card.card_mode(spcm.SPC_REP_STD_DDS)

    # Setup the card
    channels = spcm.Channels(card) # enable all channels
    channels.enable(True)
    channels.output_load(50 * units.ohm)
    channels.amp(1000 * units.mV)
    card.write_setup() # IMPORTANT! this turns on the card's system clock signals, that are required for DDS to work
    
    # Setup DDS
    dds = spcm.DDS(card, channels=channels)
    dds.reset()

    # Start the test
    num_cores = 5
    for core in dds[0:5]:
        core.amp(40 * units.percent / num_cores)
        core.freq(80 * units.MHz + int(core) * 1 * units.MHz)
        core.phase(180*np.random.rand()*units.degrees)
    dds.exec_at_trg()
    dds.write_to_card()

    # Start command including enable of trigger engine
    card.start(spcm.M2CMD_CARD_ENABLETRIGGER, spcm.M2CMD_CARD_FORCETRIGGER)

    input("Press Enter to Exit")


In [21]:
import spcm
from spcm import units


card : spcm.Card
# with spcm.Card('/dev/spcm0') as card:                         # if you want to open a specific card
# with spcm.Card('TCPIP::192.168.1.10::inst0::INSTR') as card:  # if you want to open a remote card
# with spcm.Card(serial_number=12345) as card:                  # if you want to open a card by its serial number
with spcm.Card(card_type=spcm.SPCM_TYPE_AO) as card:             # if you want to open the first card of a specific type

    # setup card for DDS
    card.card_mode(spcm.SPC_REP_STD_DDS)

    # Setup the card
    channels = spcm.Channels(card)
    channels.enable(True)
    channels.output_load(50 * units.ohm)
    channels.amp(0.1 * units.V)
    card.write_setup() # IMPORTANT! this turns on the card's system clock signals, that are required for DDS to work
    
    # Setup DDS
    dds = spcm.DDS(card, channels=channels)
    dds.reset()

    # Start the DDS test
    # Timer changes every 2.0 seconds
    period_s = 2.0 * units.s
    dds.trg_src(spcm.SPCM_DDS_TRG_SRC_TIMER)
    dds.trg_timer(period_s)
    # For slow ramps only change the value every 1000 steps
    dds.freq_ramp_stepsize(1000)
    dds.amp_ramp_stepsize(1000)

    # Create one carrier and keep on for 2 seconds
    dds[0].amp(40 * units.percent)
    dds[0].freq(5 * units.MHz) # 5 MHz
    dds.exec_at_trg()

    # Ramp the frequency of the carrier
    dds[0].frequency_slope(5 * units.MHz / units.s) # 5 MHz/s
    dds.exec_at_trg()

    # Stop frequency ramp
    dds[0].frequency_slope(0)
    dds[0].freq(15 * units.MHz) # 15 MHz
    dds.exec_at_trg()

    # Ramp the amplitude of the carrier
    dds[0].amplitude_slope(-39 * units.percent / period_s) # 1/s
    dds.exec_at_trg()

    # Stop amplitude ramp
    dds[0].amplitude_slope(0)
    dds[0].amp(1 * units.percent)
    dds.exec_at_trg()

    # Write the list of commands to the card
    dds.write_to_card()

    # Start command including enable of trigger engine
    card.start(spcm.M2CMD_CARD_ENABLETRIGGER, spcm.M2CMD_CARD_FORCETRIGGER)

    input("Press Enter to Exit")

In [ ]:
import spcm
from spcm import units


card : spcm.Card
# with spcm.Card('/dev/spcm0') as card:                         # if you want to open a specific card
# with spcm.Card('TCPIP::192.168.1.10::inst0::INSTR') as card:  # if you want to open a remote card
# with spcm.Card(serial_number=12345) as card:                  # if you want to open a card by its serial number
with spcm.Card(card_type=spcm.SPCM_TYPE_AO) as card:             # if you want to open the first card of a specific type

    # setup card for DDS
    card.card_mode(spcm.SPC_REP_STD_DDS)

    # Setup the card
    channels = spcm.Channels(card)
    channels.enable(True)
    channels.output_load(50 * units.ohm)
    channels.amp(1 * units.V)
    card.write_setup() # IMPORTANT! this turns on the card's system clock signals, that are required for DDS to work
    
    # Setup DDS
    dds = spcm.DDS(card, channels=channels)
    dds.reset()

    # Start the DDS test
    num_cores = len(dds)
    # 5 to 15 MHz
    first_init_freq_Hz  = 5.0 * units.MHz
    delta_init_freq_Hz  = 500 * units.kHz
    # 8 to 12 MHz
    first_final_freq_Hz = 8.0 * units.MHz
    delta_final_freq_Hz = 200 * units.kHz

    # STEP 0 - Initialize frequencies
    dds.trg_timer(2.0 * units.s)
    dds.trg_src(spcm.SPCM_DDS_TRG_SRC_TIMER)
    for core in dds:
        core.amp(45 * units.percent / num_cores)
        core.freq(first_init_freq_Hz + int(core) * delta_init_freq_Hz)
    dds.exec_at_trg()
    dds.write_to_card()

    # STEP 1 - Start the ramp
    period_s = 5.0 * units.s
    dds.trg_timer(period_s)
    for core in dds:
        core.frequency_slope((first_final_freq_Hz - first_init_freq_Hz + int(core) * (delta_final_freq_Hz  - delta_init_freq_Hz)) / period_s)
    dds.exec_at_trg()
    
    # STEP 2 - Stop the ramp
    for core in dds:
        core.frequency_slope(0) # Hz/s
        core.freq(first_final_freq_Hz + int(core) * delta_final_freq_Hz)
    dds.exec_at_trg()
    dds.write_to_card()

    # Start command including enable of trigger engine
    card.start(spcm.M2CMD_CARD_ENABLETRIGGER, spcm.M2CMD_CARD_FORCETRIGGER)

    input("Press Enter to Exit")

In [ ]:
import spcm
from spcm import units


card : spcm.Card
# with spcm.Card('/dev/spcm0') as card:                         # if you want to open a specific card
# with spcm.Card('TCPIP::192.168.1.10::inst0::INSTR') as card:  # if you want to open a remote card
# with spcm.Card(serial_number=12345) as card:                  # if you want to open a card by its serial number
with spcm.Card(card_type=spcm.SPCM_TYPE_AO) as card:             # if you want to open the first card of a specific type

    # setup card for DDS
    card.card_mode(spcm.SPC_REP_STD_DDS)

    # Setup the card
    channels = spcm.Channels(card)
    channels.enable(True)
    channels.output_load(50 * units.ohm)
    channels.amp(0.1 * units.V)
    card.write_setup() # IMPORTANT! this turns on the card's system clock signals, that are required for DDS to work
    
    # Setup DDS
    dds = spcm.DDS(card, channels=channels)
    dds.reset()
    dds.cores_on_channel(1, 
                             spcm.SPCM_DDS_CORE8,  spcm.SPCM_DDS_CORE9,  spcm.SPCM_DDS_CORE10, spcm.SPCM_DDS_CORE11, # Flex core block 8 - 11
                             spcm.SPCM_DDS_CORE20)
    dds[0].amp(100 * units.mV)
    dds[0].freq(100 * units.MHz)
    dds[0].phase(20 * units.degrees)
    # Read back the exact frequency
    freq = dds[0].get_freq(return_unit=units.MHz)
    amp = dds[0].get_amp(return_unit=units.dBm)
    phase = dds[0].get_phase(return_unit=units.rad)
    print(f"Generated signal frequency: {freq} and amplitude: {amp} and phase: {phase}")
    
    dds.exec_at_trg()
    dds.write_to_card()

    # Start command including enable of trigger engine
    card.start(spcm.M2CMD_CARD_ENABLETRIGGER, spcm.M2CMD_CARD_FORCETRIGGER)

    input("Press Enter to Exit")

KeyboardInterrupt: Interrupted by user